In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

In [12]:
black_soil = os.path.join('../input/soil-types/Soil types/Black Soil')
cinder_soil = os.path.join('../input/soil-types/Soil types/Cinder Soil')
laterite_soil = os.path.join('../input/soil-types/Soil types/Laterite Soil')
peat_soil = os.path.join('../input/soil-types/Soil types/Peat Soil')
yellow_soil = os.path.join('../input/soil-types/Soil types/Yellow Soil')
        

In [13]:
dataset=os.path.join('../input/soil-types/Soil types')

In [14]:
a, b, c, d, e = len(black_soil), len(cinder_soil), len(laterite_soil), len(peat_soil), len(yellow_soil)
total=a+b+c+d+e
print("Total images: ", total)

In [15]:
image_size = 220
batch_size = 10
target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)

In [16]:
from skimage.io import imread, imshow
from skimage.transform import resize

In [17]:
soil_type=["Black Soil","Cinder Soil","Laterite Soil","Peat Soil","Yellow Soil"]
fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize = (20,20))
for i in range(0,5):
    img  = imread('../input/soil-types/Soil types/'+soil_type[i]+'/10.jpg')
    img  = resize(img, (220,220))
    ax[i].imshow(img)
    ax[i].axis('off')

In [18]:
# from sklearn.model_selection import train_test_split
# training_data, testing_data = train_test_split(dataset, test_size=0.1, random_state=25)

In [18]:
train_datagen = ImageDataGenerator(rescale=1/255,rotation_range=30,
                                   horizontal_flip=False,
                                   vertical_flip=True,
                                   brightness_range=(0.2, 0.5),
                                   width_shift_range=0.3,
                                   height_shift_range=0.1,
                                   zoom_range=0.3,

                                   fill_mode='nearest')

# test_datagen  = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory(
        '../input/soil-types/Soil types/', 
        target_size=(image_size, image_size),
        batch_size = batch_size,
        classes = [ 'Black Soil','Cinder Soil', 'Laterite Soil','Peat Soil','Yellow Soil'],
       class_mode='categorical')

# test_datset = test_datagen.flow_from_directory(
#     '../input/soil-classification-image-data/Soil_Dataset/Test',
#     target_size = (224,224),
#     class_mode = 'categorical',
#     batch_size = 64)


In [19]:
for image_batch, label_batch in train_generator:
  break
  image_batch.shape, label_batch.shape

In [20]:
print (train_generator.class_indices)

In [71]:
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(220, 220, 3)),
    tf.keras.layers.MaxPooling2D(3, 3),

    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(3,3),

    
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(5, activation='softmax')
])



In [72]:
model.summary()

In [73]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer= RMSprop(learning_rate=0.001),
              metrics=['acc'])

In [74]:
total_sample = train_generator.n
n_epochs = 30

In [75]:
history = model.fit(
        train_generator, 
        steps_per_epoch = int(total_sample/batch_size),  
        epochs = n_epochs)

In [77]:
plt.figure(figsize=(7,4))
plt.plot([i+1 for i in range(n_epochs)],history.history['acc'],'-o',c='k',lw=2,markersize=9)
plt.grid(True)
plt.title("Training accuracy with epochs\n",fontsize=18)
plt.xlabel("Training epochs",fontsize=15)
plt.ylabel("Training accuracy",fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

**Some ways to save the model...**

In [78]:
model.save('my_model.h5')